In [3]:
import sys
sys.path += ['..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import Iterable, Tuple, List
from tqdm import tqdm

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def prepare_parquets(logpath):
    logpath = Path(logpath)
    logger_set = LoggerSet(logpath, overwrite_ok=False)
    logger_set.export_to_parquet()

def get_frame_idx(camera_df, angular_speed_control_df, time_offset_ms=57):
    camera_df = camera_df.copy()
    angular_speed_control_df = angular_speed_control_df.copy()
        
    camera_df['time_expected'] = camera_df['time'] + pd.to_timedelta(time_offset_ms, unit='ms')
    angular_speed_control_df['frame_idx'] = camera_df['time_expected'].searchsorted(angular_speed_control_df['time_AngularSpeedControl'], side='right')

    return angular_speed_control_df

def stack_frames(gen: Iterable[Tuple[int, np.ndarray]]):
    frames = []
    for idx, img in gen:
        frames.append(img)
    return np.stack(frames, axis=0)

def prep_for_session(logpath, time_offset_ms=100):

    logpath = Path(logpath)
    prepare_parquets(logpath)
    camera_df =  pd.read_parquet(logpath/'PicameraV2.parquet')
    control_df = pd.read_parquet(logpath/'AngularSpeedControlV2.parquet')

    control_df = get_frame_idx(camera_df, control_df, time_offset_ms=time_offset_ms)

    stacked_frames = stack_frames(get_frame_iterator(logpath/"PicameraV2/video"))

    return stacked_frames, control_df

In [5]:
def concatenate_multiple_sessions(data_of_sessions: List[Tuple[np.ndarray, pd.DataFrame]]):
    """
    MODIFY THE DATAFRAME *IN PLACE* as a side effect
    """
    frame_set_list = []
    df_list = []
    total_frames = 0
    for frames, df in data_of_sessions:
        df['total_frame_idx'] = df['frame_idx'] + total_frames

        total_frames += len(frames)
        df_list.append(df)
        frame_set_list.append(frames)

    return np.concatenate(frame_set_list, axis=0), pd.concat(df_list)

def load_multiple_session(session_paths):
    frames, dfs = [], []
    for logpath in tqdm(session_paths):
        frame, df = prep_for_session(logpath)
        frames.append(frame)
        dfs.append(df)
    
    return concatenate_multiple_sessions(zip(frames, dfs))

def prepare_to_tensorflow(frames, df):

    df = df.query('speed != 0')

    frames_reindexed = frames[df['frame_idx']]


    return frames_reindexed, df[['angular_velocity', 'left', 'right', 'speed']].values/100

frames, df = load_multiple_session(['./data/2024-05-31 20:28:39.871348/', './data/2024-05-31 20:44:54.302121/'])
data = prepare_to_tensorflow(frames, df)


100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.43s/it]


In [ ]:
frames[0]

In [11]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

imggen = ImageDataGenerator(
        rescale=1/255
    ).flow(*data)

In [7]:

def get_model(lr=.1):
    tf.keras.backend.clear_session()
    image_shape = 64, 114, 3
    
    model = keras.Sequential([
        keras.layers.InputLayer(image_shape), 
        keras.layers.Conv2D(16, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Conv2D(32, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(64, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Conv2D(64, 3, activation='relu'), 
        keras.layers.MaxPooling2D(),
        
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'), 
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(4), 
    ])

    optimiser = keras.optimizers.Adam(lr)
    model.compile(optimizer=optimiser, loss='Huber', metrics=['MAE'])

    return model 

model = get_model()
model.summary()

2024-06-01 11:10:57.972151: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-01 11:10:57.992507: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-01 11:10:57.992801: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 112, 16)    │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 56, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 54, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 27, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 12, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 10, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,076 (590.14 KB)

 Trainable params: 151,076 (590.14 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.fit(imggen, epochs=30)

Epoch 1/30


TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.